# News API

A Python client for the [News API](https://newsapi.org/docs/).


## License

Provided under [MIT License](https://github.com/mattlisiv/newsapi-python/blob/master/LICENSE.txt) by Matt Lisivick.

```
THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS
FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR
COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER
IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN
CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.
```


## General

This is a Python client library for [News API V2](https://newsapi.org/).
The functions and methods for this library should mirror the
endpoints specified by the News API [documentation](https://newsapi.org/docs/endpoints).


## Installation

Installation for the package can be done via `pip`:

In [26]:
!pip install newsapi-python

## Usage

After installation, import the client class into your project:

In [27]:
from newsapi import NewsApiClient

Initialize the client with your API key:

In [28]:
import os

api = NewsApiClient(api_key=os.environ['NEWSAPI_ORG'])


### Endpoints

An instance of `NewsApiClient` has three instance methods corresponding to three News API endpoints.

#### Top Headlines

Use `.get_top_headlines()` to pull from the [`/top-headlines`](https://newsapi.org/docs/endpoints/top-headlines) endpoint:

In [29]:
api.get_top_headlines(sources='bbc-news')

{'status': 'ok',
 'totalResults': 10,
 'articles': [{'source': {'id': 'bbc-news', 'name': 'BBC News'},
   'author': 'BBC News',
   'title': 'Gwyneth Paltrow cleared in ski crash trial',
   'description': 'The Oscar winner has won her high-profile case over a 2016 accident in Utah.',
   'url': 'http://www.bbc.co.uk/news/world-us-canada-65132827',
   'urlToImage': 'https://ichef.bbci.co.uk/news/1024/branded_news/6A92/production/_129228272_gprtop.jpg',
   'publishedAt': '2023-03-30T23:07:21.3630713Z',
   'content': 'A jury has found US actress Gwyneth Paltrow was not at fault for a 2016 ski crash at a resort in Utah.\r\nRetired optometrist Terry Sanderson, 76, had said Paltrow crashed into him, leaving him with li… [+402 chars]'},
  {'source': {'id': 'bbc-news', 'name': 'BBC News'},
   'author': 'BBC News',
   'title': 'Donald Trump indictment: Ex-US president to be charged over hush money',
   'description': 'Mr Trump, the first serving or former US president to face a criminal charge, d

#### Everything

Use `.get_everything()` to pull from the [`/everything`](https://newsapi.org/docs/endpoints/everything) endpoint:

In [30]:
api.get_everything(q='bitcoin')

{'status': 'ok',
 'totalResults': 13165,
 'articles': [{'source': {'id': 'google-news', 'name': 'Google News'},
   'author': None,
   'title': 'Bitcoin, USDC stablecoin rally after US intervenes on SVB - Reuters.com',
   'description': 'Bitcoin, USDC stablecoin rally after US intervenes on SVB\xa0\xa0Reuters.com',
   'url': 'https://consent.google.com/ml?continue=https://news.google.com/rss/articles/CBMiZGh0dHBzOi8vd3d3LnJldXRlcnMuY29tL3RlY2hub2xvZ3kvYml0Y29pbi11c2RjLXN0YWJsZWNvaW4tcmFsbHktYWZ0ZXItdXMtaW50ZXJ2ZW5lcy1zdmItMjAyMy0wMy0xMy_SAQA?oc%3D5&gl=FR&hl=en-US&cm=2&pc=n&src=1',
   'urlToImage': None,
   'publishedAt': '2023-03-13T00:55:00Z',
   'content': 'We use cookies and data to<ul><li>Deliver and maintain Google services</li><li>Track outages and protect against spam, fraud, and abuse</li><li>Measure audience engagement and site statistics to unde… [+1131 chars]'},
  {'source': {'id': 'google-news', 'name': 'Google News'},
   'author': None,
   'title': 'Bitcoin steadies near lo

#### Sources

Use `.get_sources()` to pull from the [`/sources`](https://newsapi.org/docs/endpoints/sources) endpoint:

In [31]:
api.get_sources()

{'status': 'ok',
 'sources': [{'id': 'abc-news',
   'name': 'ABC News',
   'description': 'Your trusted source for breaking news, analysis, exclusive interviews, headlines, and videos at ABCNews.com.',
   'url': 'https://abcnews.go.com',
   'category': 'general',
   'language': 'en',
   'country': 'us'},
  {'id': 'abc-news-au',
   'name': 'ABC News (AU)',
   'description': "Australia's most trusted source of local, national and world news. Comprehensive, independent, in-depth analysis, the latest business, sport, weather and more.",
   'url': 'http://www.abc.net.au/news',
   'category': 'general',
   'language': 'en',
   'country': 'au'},
  {'id': 'aftenposten',
   'name': 'Aftenposten',
   'description': 'Norges ledende nettavis med alltid oppdaterte nyheter innenfor innenriks, utenriks, sport og kultur.',
   'url': 'https://www.aftenposten.no',
   'category': 'general',
   'language': 'no',
   'country': 'no'},
  {'id': 'al-jazeera-english',
   'name': 'Al Jazeera English',
   'descr

## For Windows users printing to _cmd_ or _powershell_

You will encounter an error if you attempt to print the .json() object to the command line. This is because the '{', '}' curly braces to be printed to the console.
This becomes especially annoying if developers wish to get 'under the hood'.

Here is the error:

```
UnicodeEncodeError: 'charmap' codec can't encode character '\u2019' in position 1444: character maps to <undefined>
```

This can be fixed by:
    - installing 'win-unicode-console'
        `py -mpip install win-unicode-console`
    - then running it while calling your python script...
        `py -mrun myPythonScript.py`

Another option is hardcoding your console to only print in utf-8. This is a bad idea, as it could ruin many other scripts and/or make errors MUCH more difficult to track.
[More information](https://stackoverflow.com/questions/5419/python-unicode-and-the-windows-console/32176732#32176732).


## Showtime - Let's get some news but using it together with OpenCV and Pandas


In [32]:
import cv2
import pandas as pd
import numpy as np

In [33]:
# init
api = NewsApiClient(api_key=os.environ['NEWSAPI_ORG'])

In [34]:
def get_top_headlines():
    response = api.get_top_headlines(
        sources='bbc-news, cnn, fox-news, google-news, the-new-york-times, the-wall-street-journal, the-washington-post, time, usa-today, wired'
    )
    return response


top_headlines = get_top_headlines()

In [35]:
print(
    """
    Top Headlines
    -------------
    {}
    """.format(top_headlines)
)


    Top Headlines
    -------------
    {'status': 'ok', 'totalResults': 90, 'articles': [{'source': {'id': 'bbc-news', 'name': 'BBC News'}, 'author': 'BBC News', 'title': 'Gwyneth Paltrow cleared in ski crash trial', 'description': 'The Oscar winner has won her high-profile case over a 2016 accident in Utah.', 'url': 'http://www.bbc.co.uk/news/world-us-canada-65132827', 'urlToImage': 'https://ichef.bbci.co.uk/news/1024/branded_news/6A92/production/_129228272_gprtop.jpg', 'publishedAt': '2023-03-30T23:07:21.3630713Z', 'content': 'A jury has found US actress Gwyneth Paltrow was not at fault for a 2016 ski crash at a resort in Utah.\r\nRetired optometrist Terry Sanderson, 76, had said Paltrow crashed into him, leaving him with li… [+402 chars]'}, {'source': {'id': 'bbc-news', 'name': 'BBC News'}, 'author': 'BBC News', 'title': 'Donald Trump indictment: Ex-US president to be charged over hush money', 'description': 'Mr Trump, the first serving or former US president to face a criminal ch

In [36]:
def news_to_df(news):
    # map all the articles to a list
    articles = list(map(lambda x: x['title'], news['articles']))

    # create a dataframe
    df = pd.DataFrame(articles, columns=['title'])

    # add a column for the length of the title
    df['title_length'] = df['title'].apply(lambda x: len(x))

    # add a column for the number of words in the title
    df['title_words'] = df['title'].apply(lambda x: len(x.split(' ')))

    return df

In [37]:
df = news_to_df(top_headlines)

In [38]:
display(df)

,title,title_length,title_words
0,Gwyneth Paltrow cleared in ski crash trial,42,7
1,Donald Trump indictment: Ex-US president to be...,70,11
2,"By indicting Trump, Alvin Bragg restores our f...",69,13
3,Gwyneth Paltrow wins ski crash lawsuit,38,6
4,Trump says DA Bragg's 'obsession' with trying ...,104,16
5,"Minnesota train derailment, ethanol fire renew...",82,12
6,Gwyneth Paltrow trial verdict due after ski-cr...,57,9
7,"Top House, Senate Republicans outraged about T...",95,14
8,"Dems react: Schiff calls news 'sobering,' Wate...",93,15
9,Trump indictment follows 50 years of investiga...,65,10


In [61]:
!pip install opencv-python
!pip install mediapipe
!pip install screeninfo
!pip install cvzone

  Using cached cvzone-1.5.6.tar.gz (12 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for cvzone: filename=cvzone-1.5.6-py3-none-any.whl size=18760 sha256=fd332a7d08377e06aa8b7373e51951966a334b9b86736f9b5545f9b42495b8bd
  Stored in directory: c:\users\yilma\appdata\local\pip\cache\wheels\d8\85\f1\1756f9e009d280be742fb20dd5087c60c2f7f0279964934375
Successfully built cvzone


In [65]:
import cv2
import mediapipe as mp
import cvzone
import numpy as np
import screeninfo

from cvzone.HandTrackingModule import HandDetector

In [71]:
# Using OpenCV to display the image
cap = cv2.VideoCapture(0)

cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 1024)
cap.set(cv2.CAP_PROP_FPS, 60)

# text as one line string
text = ' '.join(df['title'].tolist())

# add '###' between each title
text = ' ... | '.join(df['title'].tolist())

# font
font = cv2.FONT_HERSHEY_SIMPLEX

detector = HandDetector(detectionCon=0.8, maxHands=2)

while True:

    # get the frame
    ret, frame = cap.read()

    if not ret:
        break

    # add the channel name to the top left corner
    cv2.putText(frame, 'AI News', (10, 30), font, 1, (0, 255, 0), 2, cv2.LINE_AA)

    # choose a random RGB color
    # add news to the right bottom center
    # if the text starts with ### then change the color

    prev_color = (255,255, 255)
    cv2.putText(frame, text, (0, int(frame.shape[0] - 10)), font, 1, prev_color, 2, cv2.LINE_AA)

    hands = detector.findHands(frame, draw=False)

    # print number of hands raised in the top right corner
    cv2.putText(frame, f'Hands: {len(hands)}', (int(frame.shape[1] - 200), 30), font, 1, (0, 255, 0), 2, cv2.LINE_AA)

    # simulate a news ticker by moving the text to the left slowly
    # if I hold both my hand in front of the camera then stop the news feed
    if len(hands) == 2:
        pass
    else:
        text = text[1:] + text[0]

    # show the frame
    cv2.imshow('AI News', frame)

    # wait for the 'q' key to be pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

    # if 's' is pressed, save the image
    if cv2.waitKey(1) & 0xFF == ord('s'):
        cv2.imwrite('news.jpg', frame)

cap.release()
cv2.destroyAllWindows()

In [ ]:
!pip install matplotlib

In [6]:
# Using Matplotlib to display the image

import matplotlib.pyplot as plt

# get the image
img = cv2.imread('news.jpg')

# show the image
plt.imshow(img)


NameError: name 'df' is not defined